Retrieve all the approved drugs from ChEMBL as well as the date of approval and the SMILES.


In [4]:
!pip install datamol
!pip install chembl_webresource_client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 863.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 3.3 MB/s eta 0:00:00


In [5]:
pip install chembl_webresource_client

In [6]:
import pandas as pd
import datamol as dm

from chembl_webresource_client.new_client import new_client as client


In [8]:
# First, we retrieve the ChEMBL IDs for all the approved drugs (max_phase=4)
mol_ids = client.molecule.filter(max_phase=4).only(["molecule_chembl_id"])
mol_ids = pd.DataFrame(mol_ids)

len(mol_ids)


4384

In [9]:
# Now retrieve other columns of interest

columns = ["molecule_chembl_id", "pref_name", "first_approval", "molecule_structures", "molecule_type"]


def _get_mol(molecule_chembl_id):
    mols = client.molecule.filter(molecule_chembl_id=molecule_chembl_id).only(columns)
    assert len(mols) == 1
    mol = mols[0]

    if mol.get("molecule_structures") is not None and "canonical_smiles" in mol.get(
        "molecule_structures", []
    ):
        mol["smiles"] = mol["molecule_structures"]["canonical_smiles"]

    if "molecule_structures" in mol:
        del mol["molecule_structures"]

    return pd.Series(mol)


mols = dm.parallelized(
    _get_mol,
    mol_ids["molecule_chembl_id"],
    n_jobs=256,
    scheduler="threads",
    progress=True,
)
mols = pd.DataFrame(mols)

mols.head()


  0%|          | 0/4384 [00:00<?, ?it/s]

ERROR:requests_cache.backends.sqlite:database is locked
ERROR:requests_cache.backends.sqlite:database is locked
ERROR:requests_cache.backends.sqlite:database is locked
ERROR:requests_cache.backends.sqlite:database is locked


,first_approval,molecule_chembl_id,molecule_type,pref_name,smiles
0,1976.0,CHEMBL2,Small molecule,PRAZOSIN,COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC
1,1984.0,CHEMBL3,Small molecule,NICOTINE,CN1CCC[C@H]1c1cccnc1
2,1990.0,CHEMBL4,Small molecule,OFLOXACIN,CC1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23
3,1964.0,CHEMBL5,Small molecule,NALIDIXIC ACID,CCn1cc(C(=O)O)c(=O)c2ccc(C)nc21
4,1965.0,CHEMBL6,Small molecule,INDOMETHACIN,COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c1ccc(Cl)cc1


In [11]:
# Let's focus on small molecules with a valid SMILES and a first approval date
mols = mols.query("molecule_type == 'Small molecule' & smiles.notna() & first_approval.notna()")
mols = mols.reset_index(drop=True)

mols

,first_approval,molecule_chembl_id,molecule_type,pref_name,smiles
0,1976.0,CHEMBL2,Small molecule,PRAZOSIN,COc1cc2nc(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC
1,1984.0,CHEMBL3,Small molecule,NICOTINE,CN1CCC[C@H]1c1cccnc1
2,1990.0,CHEMBL4,Small molecule,OFLOXACIN,CC1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)cn1c23
3,1964.0,CHEMBL5,Small molecule,NALIDIXIC ACID,CCn1cc(C(=O)O)c(=O)c2ccc(C)nc21
4,1965.0,CHEMBL6,Small molecule,INDOMETHACIN,COc1ccc2c(c1)c(CC(=O)O)c(C)n2C(=O)c1ccc(Cl)cc1
...,...,...,...,...,...
2771,2021.0,CHEMBL5315120,Small molecule,ODEVIXIBAT SESQUIHYDRATE,CCCCC1(CCCC)CN(c2ccccc2)c2cc(SC)c(OCC(=O)N[C@@...
2772,2018.0,CHEMBL5315121,Small molecule,TECOVIRIMAT MONOHYDRATE,O.O=C(NN1C(=O)[C@@H]2[C@@H]3C=C[C@@H]([C@H]4C[...
2773,2015.0,CHEMBL5315122,Small molecule,BORTEZOMIB D-MANNITOL,CC(C)C[C@H](NC(=O)[C@H](Cc1ccccc1)NC(=O)c1cncc...
2774,1996.0,CHEMBL5315124,Small molecule,LEVOFLOXACIN,C[C@H]1COc2c(N3CCN(C)CC3)c(F)cc3c(=O)c(C(=O)O)...


In [15]:
!pip install gradio cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.2/210.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12

In [16]:
# Save as Parquet
mols.to_parquet("/content/chembl_approved_drugs.parquet", index=False)

In [17]:
import pandas as pd

# Read the Parquet file
df = pd.read_parquet('/content/chembl_approved_drugs.parquet')

# Convert to CSV
df.to_csv('output_file.csv', index=False)

In [18]:
import pandas as pd
import datamol as dm
import gradio as gr
import cohere
from chembl_webresource_client.new_client import new_client as client
from rdkit import Chem
from rdkit.Chem import Draw

# Set up Cohere API
cohere_api_key = "OIgOnN9bV378w7IJyp68yTYH7XiGs3PpqJAKAhbI"
co = cohere.Client(cohere_api_key)

# Define the columns of interest
columns = ["molecule_chembl_id", "pref_name", "first_approval", "molecule_structures", "molecule_type"]

# Function to fetch molecule data by ChEMBL ID or SMILES
def fetch_molecule_data(chembl_id=None, smiles=None):
    print(f"Received ChEMBL ID: {chembl_id}, SMILES: {smiles}")
    mols = []
    if chembl_id:
        mols = client.molecule.filter(molecule_chembl_id=chembl_id).only(columns)
    elif smiles:
        mols = client.molecule.filter(molecule_structures__canonical_smiles=smiles).only(columns)

    print(f"Retrieved Molecule Data: {mols}")

    if len(mols) == 1:
        mol = mols[0]
        print(f"Fetched Molecule: {mol}")
        return mol.get("pref_name", "Name not found"), mol
    return "Name not found", None

# Function to query Cohere for detailed drug target information
def get_drug_target_info(chembl_id, smiles, pref_name):
    prompt = f"""
    You are an expert in drug discovery. Based on the following information:
    - ChEMBL ID: {chembl_id}
    - SMILES: {smiles}
    - Preferred Name: {pref_name}

    Provide a detailed summary of the drug's known biological targets and its mechanism of action.
    Provide approaches towards the synthesis of new compounds approaching novel drugs for multi targets, give chemical and physical properties of the drug as well.
    Provide target protein structures and formulas for the drug.
    """

    response = co.generate(
        model="command",
        prompt=prompt,
        max_tokens=300,
        temperature=0.5,
        stop_sequences=["--"]
    )
    return response.generations[0].text.strip()

# Function to generate molecular structure image
def generate_structure(smiles):
    if smiles:
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            img = Draw.MolToImage(mol)
            return img
    return None

# Gradio function that integrates the molecule retrieval and Cohere integration
def gradio_interface(chembl_id, smiles):
    # Fetch molecule data
    pref_name, mol_data = fetch_molecule_data(chembl_id=chembl_id, smiles=smiles)

    if mol_data:
        # Get drug target information using Cohere
        drug_info = get_drug_target_info(chembl_id, smiles, pref_name)
        # Generate molecular structure image
        structure_img = generate_structure(smiles)
        return pref_name, drug_info, structure_img
    else:
        return "No data found", "No target information available.", None

# Create Gradio interface
iface = gr.Interface(
    fn=gradio_interface,
    inputs=[gr.Textbox(label="ChEMBL ID"), gr.Textbox(label="SMILES")],
    outputs=[gr.Textbox(label="Preferred Name"), gr.Textbox(label="Drug Target Information"), gr.Image(label="Molecular Structure")],
    title="DRUG__DISCOVER",
    description="This app retrieves the preferred name, provides detailed drug target information using the Cohere API, and shows the molecular structure based on the target compund."
)

# Launch the app
iface.launch()


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9d235e3f4516bd5363.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
